In [1]:
import re
import fnmatch
from pathlib import Path
from typing import List, Set

In [ ]:
def find_run_numbers(directory: Path, reg_expression: str) -> Set[int]:
    """
    Finds all run numbers in a directory based off of a regular expression.
    Supports only one capture group in the regular expression (this should be the run number!)
    """
    run_numbers = set()
    if not directory.is_dir():
        raise NotADirectoryError(f"Directory not found for: {directory}")
    
    expression = re.compile(reg_expression)
    for file in directory.iterdir():
        if not file.is_file():
            continue

        if match := expression.match(file.name):
            if len(match.groups()) != 1:
                raise ValueError(f"Please provide only one capture group in regular expresion, {len(match.groups())} found. Remember in regex capture groups are denoted by parenthesis ()") 
            run_num_raw = match.group(1)
            if run_num_raw.isdigit():
                run_numbers.add(int(run_num_raw))
            else:
                raise ValueError(f"Your inputted regular expression did not output an integer. Output: {run_num_raw}")
    return run_numbers


binaries = find_run_numbers(
    Path("/home/etl/Test_Stand/ETL_TestingDAQ/Test_Beam_Data/DebugTB/daq/etroc_binaries"),
    r"output_run_(\d+)_rb0.dat"
)

hitmaps = find_run_numbers(
    Path("/home/etl/Test_Stand/ETL_TestingDAQ/Test_Beam_Data/DebugTB/watchdog/etroc_hitmaps"),
    r"output_run_(\d+)_rb0.png"
)

# return all non matching elements 

missing_runs = list(set(binaries)-set(hitmaps))

for run in missing_runs:
    # make hitmaps at location
    ...


38


In [2]:
startup_prompt = """
Please choose one of the following options by inputting 1,2 or 3:
1. Start from last run
2. Custom run start, choosing this will start watchdog from this run number
3. Same as 2, but you can choose a run to stop the watchdog at.                                                               
"""
run_start = None
while not run_start:
    startup_input = input(startup_prompt).strip()
    if startup_input == "1":
        print("Sorry this is not implemented")
    elif startup_input == "2":
        run_start_raw = input("What is your run number?").strip()
        if not run_start_raw.isdigit():
            print(f"Your chosen starting run number is not a number: {run_start_raw}")
        run_start = int(run_start_raw)
    elif startup_input == "3":
        print("Sorry this is not implemented")
    else:
        print(f"Invalid input ({startup_input}), try again")

print(f"Watchdog starting at run: {run_start}")


Invalid input (a), try again
Sorry this is not implemented
Watchdog starting at run: 3
